In [1]:
import sys, os
from pathlib import Path
WORKDIR = Path(os.getcwd())
os.chdir(WORKDIR)
os.environ['WORKDIR'] = str(WORKDIR)

In [2]:
!sudo apt install -yy htop ninja-build
#!pip install pytorch-lightning
!conda install pytorch-lightning=1 opencv

Reading package lists... Done
Building dependency tree       
Reading state information... Done
htop is already the newest version (2.2.0-1+b1).
ninja-build is already the newest version (1.8.2-1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.
Solving environment: - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | done

# All requested packages already installed.



In [7]:
!git clone https://github.com/amikhalev/TinyHITNet.git TinyHITNet || true
HITNET = WORKDIR / 'TinyHITNet'
sys.path.append(str(HITNET))

fatal: destination path 'TinyHITNet' already exists and is not an empty directory.


In [ ]:
!TORCH_CUDA_ARCH_LIST="3.7+PTX" pip install ./TinyHITNet/ext_op/

Processing ./TinyHITNet/ext_op
  Preparing metadata (setup.py) ... - done
  Created wheel for HitnetModule: filename=HitnetModule-0.0.0-cp37-cp37m-linux_x86_64.whl size=2399331 sha256=e25c2debfce7d7d7a43a3fef95a4216ee95a0d2ea707eb794fa3cce01f9c34e0
  Stored in directory: /tmp/pip-ephem-wheel-cache-d0f0nyju/wheels/7c/53/75/d76e82895fa5974e0be9f13d627c1832ea50cf17245149ddbc
Successfully built HitnetModule


In [ ]:
!nvidia-smi

Sun Mar 13 00:54:38 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.73.01    Driver Version: 460.73.01    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    53W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [10]:
!DIR=kitty2015; mkdir -p $DIR; cd $DIR; \
wget -c https://s3.eu-central-1.amazonaws.com/avg-kitti/data_scene_flow.zip; \
unzip -qn data_scene_flow.zip
KITTY2015 = WORKDIR / "kitty2015"

--2022-03-13 06:03:42--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_scene_flow.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.170.137
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.170.137|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [11]:
!DIR=kitty2012; mkdir -p $DIR; cd $DIR; \
wget -c https://s3.eu-central-1.amazonaws.com/avg-kitti/data_stereo_flow.zip; \
unzip -qn data_stereo_flow.zip
KITTY2012 = WORKDIR /"kitty2012"

--2022-03-13 06:03:45--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_stereo_flow.zip
Resolving s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)... 52.219.170.137
Connecting to s3.eu-central-1.amazonaws.com (s3.eu-central-1.amazonaws.com)|52.219.170.137|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.



In [ ]:
from preprocess import plane_fitting_kitti2012, plane_fitting_kitti2015
import multiprocessing as mp
import tqdm

def preprocess_main(root, list_path, module, n_gpu=1):
    root = Path(root)

    with open(list_path, "rt") as fp:
        file_list = [Path(line.strip()) for line in fp]

    lock_list = [mp.Lock() for _ in range(n_gpu)]
    with mp.Pool(n_gpu, module.process_init, [lock_list, root]) as pool:
        list(tqdm.tqdm(pool.imap_unordered(module.process, file_list), total=len(file_list)))

preprocess_main(KITTY2012 / 'training', HITNET / 'lists' / 'kitti2012_train.list', plane_fitting_kitti2012)
preprocess_main(KITTY2015 / 'training', HITNET / 'lists' / 'kitti2015_train.list', plane_fitting_kitti2015)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200/200 [00:16<00:00, 12.41it/s]


In [ ]:
!cd TinyHITNet/; python3 train.py \
--log_dir logs \
--exp_name stereonet_kitti \
--model StereoNet \
--check_val_every_n_epoch 5 \
--max_steps 2000000 \
--max_disp 192 \
--max_disp_val 192 \
--optmizer Adam \
--lr 4e-4 \
--lr_decay 400000 0.25 408000 0.1 410000 0.025 \
--lr_decay_type Lambda \
--batch_size 8 \
--batch_size_val 8 \
--num_workers 12 \
--num_workers_val 12 \
--data_augmentation 1 \
--data_type_train KITTI2012 KITTI2015 \
--data_root_train {KITTY2012}/training {KITTY2015}/training \
--data_list_train lists/kitti2012_train170.list lists/kitti2015_train180.list \
--data_size_train 1152 320 \
--data_type_val KITTI2015 \
--data_root_val {KITTY2015}/training \
--data_list_val lists/kitti2015_val20.list \
--data_size_val 1242 375 \
--gpus -1 \
--accelerator gpu


Global seed set to 2021
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name         | Type             | Params
--------------------------------------------------
0 | model        | StereoNet        | 534 K 
1 | train_metric | MetricCollection | 0     
2 | val_metric   | MetricCollection | 0     
--------------------------------------------------
534 K     Trainable params
0         Non-trainable params
534 K     Total params
2.139     Total estimated model params size (MB)
Global seed set to 2021
/opt/conda/lib/python3.7/site-packages/pytorch_lightning/trainer/data_loading.py:433: UserWarning: The number of training samples (43) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.
  f"The number of training samples ({self.num_training_batches}) is smaller than the logging inte